In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, date, timedelta, time
os.environ['TZ'] ='America/New_York'

import yfinance as yf

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from ib_async import *
util.startLoop()

### Data Prep for Inferencing

In [ ]:
ib = IB()
ib.connect(port=4002, clientId=0)

In [ ]:
contract = Contract(symbol='MSFT', secType='STK', exchange='SMART', currency='USD')
ib.qualifyContracts(contract)

In [ ]:
contract_vix = Contract(symbol= 'VIX', secType = 'IND',exchange = 'CBOE', currency='USD')
ib.qualifyContracts(contract_vix)

In [ ]:
contract_nas = Contract(symbol= 'TQQQ', secType = 'STK',exchange = 'SMART', currency='USD')
ib.qualifyContracts(contract_nas)

In [2]:
class Data_Request :

    """
    Returns PD data containning equity price with specific time control

    Returns:
        pd: information regarding the contract of interst.
    """
    
    def __init__(self, list_days, contract, barsize, duration, *,specific_time= None, specific_pick_time= None):

        self.list_days = list_days
        self.contract = contract
        #self.init_date = init_date
        self.barsize = barsize
        self.duration = duration
        self.specific_time = specific_time # cut off time . Time object: time(14, 30, 0)
        self.specific_pick_time= specific_pick_time

    def D_request(self):

        his_df =pd.DataFrame()
        
        for i in range(len(self.list_days)):
            init_date=self.list_days[i]
            bars = ib.reqHistoricalData(self.contract, init_date, barSizeSetting=self.barsize, durationStr=self.duration, whatToShow="TRADES", useRTH=True)
            add_his_df = pd.DataFrame(bars)
            add_his_df['date'] = pd.to_datetime(add_his_df['date'])
            print(f'retriveing data for {init_date}')

            if self.specific_time is not None:
                cut_off = datetime.combine(self.list_days[i],self.specific_time)  # need logic if specific_time is defined
                pd_cut_off = pd.to_datetime(cut_off).tz_localize('US/Eastern')
            #datetime64_eastern = pd.Timestamp(date_time_str).tz_localize('US/Eastern')
                add_his_df = add_his_df[add_his_df['date'] > pd_cut_off]
            
            if self.specific_pick_time is not None:
                pick_time = datetime.combine(self.list_days[i],self.specific_pick_time)  # need logic if specific_time is defined
                pd_pick_time = pd.to_datetime(pick_time).tz_localize('US/Eastern')
            #datetime64_eastern = pd.Timestamp(date_time_str).tz_localize('US/Eastern')
                add_his_df = add_his_df[add_his_df['date'] == pd_pick_time]

            
            his_df = pd.concat([his_df, add_his_df], ignore_index=True)
            his_df['Date_Only'] = his_df['date'].dt.date
            print(len(his_df))

        
        return his_df


In [3]:
# picking out Friday and before specific-time data

#specific_time = time(14, 30, 0)
def data_filter(df_initial, list_days, specific_time_before = None, specific_time_after= None, on_time= None):
    his_df = pd.DataFrame()
    for i in range(len(list_days)):
        init_date=list_days[i]
        if specific_time_before is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time < specific_time_before)]
           
            
        if specific_time_after is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time >= specific_time_after)]

        if on_time is not None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date) & (df_initial['date'].dt.time == on_time)]

        if (specific_time_before is None and specific_time_after is None) and on_time is None:
            add_his_df = df_initial[(df_initial['date'].dt.date == init_date)]
        
        print(len(his_df))
        print(f'retrieving {init_date}')
        per_proc = np.round(i+1/len(list_days)*100, decimals=1)
        print(f'{per_proc} percent is done')
        his_df = pd.concat([his_df, add_his_df], ignore_index=True)
        his_df.drop_duplicates(subset=None, keep='first', inplace=True)
        his_df['Date_Only'] = his_df['date'].dt.date
    return his_df

In [4]:
from datetime import date, timedelta

def calculate_business_days(start_date, end_date):
    """
    Calculates the number of business days between two dates, excluding weekends.

    Args:
        start_date (date): The start date.
        end_date (date): The end date.

    Returns:
        int: The number of business days.
    """
    business_days = 0
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() < 5:  # Monday to Friday
            business_days += 1
        current_date += timedelta(days=1)
    return business_days

# Example usage:
start_date = date(2025, 2, 1)  # April 7, 2025 (Monday)
end_date = date(2025, 4, 3)  # April 18, 2025 (Friday)
num_business_days = calculate_business_days(start_date, end_date)
print(f"Number of business days between {start_date} and {end_date}: {num_business_days}")

Number of business days between 2025-02-01 and 2025-04-03: 44


In [5]:
def get_fridays_in_range(start_date, end_date):
    """
    Returns a list of dates that are Fridays within the given date range.

    Args:
        start_date (date): The start date of the range.
        end_date (date): The end date of the range.

    Returns:
        list: A list of date objects representing Fridays within the range.
    """
    fridays = []
    all_dates = []
    current_date = start_date
    all_current_date = start_date

    # Adjust to the first Friday
    days_until_friday = (4 - current_date.weekday()) % 7
    current_date += timedelta(days=days_until_friday)

    while current_date <= end_date:
        fridays.append(current_date)
        current_date += timedelta(days=7) # Move to the next Friday

    while all_current_date <= end_date:
        all_dates.append(all_current_date)
        all_current_date += timedelta(days=1)
    
    return fridays, all_dates

# Example Usage
start_date = date(2024, 2, 1)
end_date = date(2025, 4, 1)

fridays, all_dates = get_fridays_in_range(start_date, end_date)
# removing Non-trading Holidays

fridays.remove(date(2024, 3, 29))
fridays.remove(date(2024, 11, 29))

#all_dates.remove(date(2024, 1, 15))
all_dates.remove(date(2024, 2, 19))
all_dates.remove(date(2024, 3, 29))
all_dates.remove(date(2024, 5, 27))
all_dates.remove(date(2024, 6, 19))
all_dates.remove(date(2024, 7, 4))
all_dates.remove(date(2024, 9, 2))
all_dates.remove(date(2024, 11, 28))
all_dates.remove(date(2024, 11, 29))
all_dates.remove(date(2024, 12, 25))

all_dates.remove(date(2025, 1, 1))
all_dates.remove(date(2025, 1, 20))
all_dates.remove(date(2025, 2, 17))

# Wednesday, January 1, 2025: New Year's Day
# Monday, January 20, 2025: Martin Luther King Jr. Day
# Monday, February 17, 2025: Presidents' Day
# Friday, April 18, 2025: Good Friday
# Monday, May 26, 2025: Memorial Day
# Thursday, June 19, 2025: Juneteenth National Independence Day
# Friday, July 4, 2025: Independence Day
# Monday, September 1, 2025: Labor Day
# Thursday, November 27, 2025: Thanksgiving Day
# Thursday, December 25, 2025: Christmas Day

In [6]:
biz_dates = [ d for d in all_dates if d.weekday()<5]

In [7]:
now =datetime.now()
print(now.date())

2025-04-05


### Trainning Data Set Collection ###

In [ ]:
barsize ="1 min"
duration = "1 D"
DR_stock_meta = Data_Request(biz_dates, contract, barsize, duration)
df_all_days_stock = DR_stock_meta.D_request()

In [ ]:
DR_IND_VIX = Data_Request(biz_dates, contract_vix, barsize, duration)
DR_STK_TQQQ = Data_Request(biz_dates, contract_nas, barsize, duration)
df_all_days_VIX = DR_IND_VIX.D_request()


In [ ]:
df_all_days_TQQQ = DR_STK_TQQQ.D_request()

In [ ]:
df_TQQQ = df_all_days_TQQQ

In [ ]:
df_VIX = df_all_days_VIX[(df_all_days_VIX['date'].dt.time >= time(9,30,0))].copy()

In [ ]:
df_all_days_stock.to_csv('dev_master_stock_info.csv', index=False)
df_all_days_stock.to_pickle('dev_master_stock_info.pkl')

#df_VIX.to_csv('dev_master_vix_info.csv', index=False)
#df_VIX.to_pickle('dev_master_vix_info.pkl')

#df_TQQQ.to_csv('dev_master_tqqq_info.csv', index=False)
#df_TQQQ.to_pickle('dev_master_tqqq_info.pkl')

In [9]:
df_all_days_stock = pd.read_pickle('dev_master_stock_info.pkl')
df_VIX = pd.read_pickle('dev_master_vix_info.pkl')
df_TQQQ = pd.read_pickle('dev_master_tqqq_info.pkl')

In [8]:
cut_time = time(14, 30, 0)

In [ ]:

# def calculate_rsi(prices, period=14):
#     """
#     Calculates the Relative Strength Index (RSI) for a given price series.

#     Args:
#         prices (pd.Series): A Pandas Series representing the price data.
#         period (int, optional): The lookback period for RSI calculation. Defaults to 14.

#     Returns:
#         pd.Series: A Pandas Series containing the RSI values.
#     """

#     delta = prices.diff()
#     gain = delta.where(delta > 0, 0)
#     loss = -delta.where(delta < 0, 0)

#     avg_gain = gain.rolling(window=period, min_periods=period).mean()
#     avg_loss = loss.rolling(window=period, min_periods=period).mean()

#     rs = avg_gain / avg_loss
#     rsi = 100 - (100 / (1 + rs))

#     return rsi



In [ ]:
# Example usage:
# #data = {'Close': [45, 48, 50, 47, 49, 52, 55, 53, 51, 54]}
# #df = pd.DataFrame(data)
# df = pd.DataFrame()
# rsi_period = 14 
# df_all_days_stock['RSI_14'] = calculate_rsi(df_all_days_stock['close'], period=rsi_period)
# rsi_period = 30 
# df_all_days_stock['RSI_30'] = calculate_rsi(df_all_days_stock['close'], period=rsi_period)
# rsi_period = 60 
# df_all_days_stock['RSI_60'] = calculate_rsi(df_all_days_stock['close'], period=rsi_period)

# df_all_days_stock

In [ ]:
df_filter_after.groupby('Date_Only')['open'].max()

In [ ]:
df_filter_after.groupby('Date_Only')['close'].min()

In [10]:

def data_sparcing_construc(df, sp_day, *, cut_time = None):
    #df_group = df.groupby("Date_Only").std(numeric_only=True)
    df_filter_prior = data_filter(df, sp_day, specific_time_before = cut_time )
    df_group_prior = df_filter_prior.groupby("Date_Only").std(numeric_only=True)
    
    df_filter_after = data_filter(df, sp_day, specific_time_after = cut_time )
    df_group_after = df_filter_after.groupby("Date_Only").std(numeric_only=True)


    df_group_train =pd.DataFrame()
    df_group_train['Target_std'] = pd.DataFrame(df_group_after[['open','high','low','close']].max(axis=1))
    df_group_train['Target_spread'] = df_filter_after.groupby('Date_Only')['open'].max() - df_filter_after.groupby('Date_Only')['open'].min()
    
    df_group_train['prior_std'] = pd.DataFrame(df_group_prior[['open','high','low','close']].max(axis=1))
    df_group_train['prior_Vol'] = pd.DataFrame(df_filter_prior.groupby("Date_Only").mean()['volume'])
    df_group_train['prior_spread'] = df_filter_prior.groupby('Date_Only')['open'].max() - df_filter_prior.groupby('Date_Only')['open'].min()

    df_group_train['prior_range'] =pd.DataFrame(df_filter_prior.groupby('Date_Only')['close'].max() -df_filter_prior.groupby('Date_Only')['close'].min() )
    
    df_initial = df_filter_prior.copy()
    new_time = time(cut_time.hour, cut_time.minute -1)
    df_group_train['cut_open'] = df_initial[(df_initial['date'].dt.time == new_time)]['open'].values
   # df_group_train['RSI_14'] = df_initial[df_initial['date'].dt.time == new_time]['RSI_14'].values
   # df_group_train['RSI_30'] = df_initial[df_initial['date'].dt.time == new_time]['RSI_30'].values
   # df_group_train['RSI_60'] = df_initial[df_initial['date'].dt.time == new_time]['RSI_60'].values
    
    cut_name =['cut_1hr','cut_2hr','cut_3hr']
    for i in range(3):
        print(i)
        prior = time(cut_time.hour - (i+1), cut_time.minute)
        df_group_train[cut_name[i]] = df_initial[(df_initial['date'].dt.time == new_time)]['open'].values- df_initial[(df_initial['date'].dt.time == prior)]['open'].values

    #df_group_train['Class_target'] = df_filter_after[(df_filter_after['date'].dt.time == time(15, 49, 0))]['close'].values- df_initial[(df_initial['date'].dt.time == new_time)]['open'].values
    
    return df_group_train, df_filter_prior, df_filter_after, df_group_prior, df_group_after
    

In [11]:
df_group_train, df_filter_prior, df_filter_after, df_group_prior, df_group_after = data_sparcing_construc(df_all_days_stock, all_dates, cut_time= cut_time)

0
retrieving 2024-02-01
0.2 percent is done
300
retrieving 2024-02-02
1.2 percent is done
600
retrieving 2024-02-03
2.2 percent is done
600
retrieving 2024-02-04
3.2 percent is done
600
retrieving 2024-02-05
4.2 percent is done
900
retrieving 2024-02-06
5.2 percent is done
1200
retrieving 2024-02-07
6.2 percent is done
1500
retrieving 2024-02-08
7.2 percent is done
1800
retrieving 2024-02-09
8.2 percent is done
2100
retrieving 2024-02-10
9.2 percent is done
2100
retrieving 2024-02-11
10.2 percent is done
2100
retrieving 2024-02-12
11.2 percent is done
2400
retrieving 2024-02-13
12.2 percent is done
2700
retrieving 2024-02-14
13.2 percent is done
3000
retrieving 2024-02-15
14.2 percent is done
3300
retrieving 2024-02-16
15.2 percent is done
3600
retrieving 2024-02-17
16.2 percent is done
3600
retrieving 2024-02-18
17.2 percent is done
3600
retrieving 2024-02-20
18.2 percent is done
3900
retrieving 2024-02-21
19.2 percent is done
4200
retrieving 2024-02-22
20.2 percent is done
4500
retri

ValueError: operands could not be broadcast together with shapes (289,) (291,) 

In [ ]:
df_all_days_stock

In [ ]:
df_all_days_stock.shape

In [ ]:
df_VIX.shape

In [ ]:
df_TQQQ.shape

In [ ]:
for d in biz_dates:
    for df in df_TQQQ:
        if len(df_TQQQ[df_TQQQ['Date_Only']==d]) !=390:
            print(d)

In [ ]:
df_VIX_clean = df_VIX[(df_VIX['Date_Only'] !=date(2024,7,3)) &  (df_VIX['Date_Only'] != date(2024,12,24)) & (df_VIX['Date_Only'] != date(2025,1,8)) &(df_VIX['Date_Only'] != date(2025,1,9)) ]

In [ ]:
df_stock_clean = df_all_days_stock[(df_all_days_stock['Date_Only'] !=date(2024,7,3)) &  (df_all_days_stock['Date_Only'] != date(2024,12,24)) & (df_all_days_stock['Date_Only'] != date(2025,1,8)) &(df_all_days_stock['Date_Only'] != date(2025,1,9)) ]

In [ ]:
df_TQQQ_clean = df_TQQQ[(df_TQQQ['Date_Only'] !=date(2024,7,3)) &  (df_TQQQ['Date_Only'] != date(2024,12,24)) & (df_TQQQ['Date_Only'] != date(2025,1,8)) &(df_TQQQ['Date_Only'] != date(2025,1,9)) ]

In [ ]:
df_TQQQ.shape

In [ ]:
# switching
df_VIX = pd.DataFrame()
df_VIX = df_VIX_clean.copy()

df_TQQQ = pd.DataFrame()
df_TQQQ = df_TQQQ_clean.copy()

df_all_days_stock = pd.DataFrame()
df_all_days_stock = df_stock_clean.copy()

In [ ]:
df_group_train_vix, *_  = data_sparcing_construc(df_VIX, all_dates, cut_time= cut_time)
df_group_train_tqqq, *_ = data_sparcing_construc(df_TQQQ, all_dates, cut_time= cut_time)

In [ ]:
df_group_train_tqqq

In [ ]:
df_final_jointed_vix=pd.DataFrame()
df_final_jointed_all=pd.DataFrame()

df_group_train_vix.columns =['vix_std','vix_spread','vix_prior_std','vix_Vol', 'vix_pr_spread','vix_prior_range','vix_cut_open','vix_cut_1hr','vix_cut_2hr','vix_cut_3hr']
df_final_jointed_vix = pd.concat([df_group_train,df_group_train_vix], axis=1)
df_group_train_tqqq.columns =['tqqq_std','tqqq_spread','tqqq_prior_std','tqqq_Vol','tqqq_pr_spread','tqqq_prior_range','tqqq_cut_open','tqqq_cut_1hr','tqqq_cut_2hr','tqqq_cut_3hr']
df_final_jointed_all = pd.concat([df_final_jointed_vix,df_group_train_tqqq], axis=1)
df_final_jointed_all.drop(['vix_std','vix_Vol','tqqq_std'], axis=1, inplace=True)

In [ ]:
df_final_jointed_all

In [ ]:
df_final_jointed_all.columns

In [ ]:
df_final_jointed_all.drop(['vix_spread','tqqq_spread'], axis=1, inplace =True)

In [ ]:
df_final_jointed_all.columns

In [ ]:
df_final_jointed_all.to_pickle('dev_df_final_jointed_all_train_beta.pkl')